## Linguistic Similarity and Message Preference Analysis

### Steps
1. Loading the Data
2. Preparing the Data
3. Analysis 1: Message similarity --> Message Preference
4. Analysis 2: Message similarity --> Perceived Personalization
5. Analysis 3: Message similarity --> Perceived Message Effectiveness
6. *Exploratory Analysis:* Similarity*Valence --> Message Preference
7. *Exploratory Analysis:* Message Similarity --> Perceived Soure Similarity

## 1. Loading the Data

In [ ]:
#Lets load the necessary packages and install bambi

import pandas as pd
!pip install bambi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Now we can import bambi
import bambi as bmb

In [13]:
# And upload our necessary files
from google.colab import files
uploaded = files.upload()

Saving eff_long.csv to eff_long (2).csv
Saving pp_long.csv to pp_long (2).csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving complete_df_EXPSIM4.csv to complete_df_EXPSIM4.csv


In [14]:
from google.colab import files
uploaded = files.upload()

Saving complete_df_EXPSIM4.csv to complete_df_EXPSIM4 (1).csv


In [16]:
# Now we can read them into our dataframe
complete_df = pd.read_csv("complete_df_EXPSIM4.csv")
pp = pd.read_csv("pp_long (2).csv")
eff = pd.read_csv("eff_long (2).csv")
ps = pd.read_csv("complete_wide_small.csv")

In [17]:
#Lets have a look into our data // Message preference
complete_df.head()

,Unnamed: 0,Movez_code_x,Linguistic Category,Similarity: EXP,Sex,Gender,age,FAS,Message_pref,Similarity: EXP_100
0,0,1016110.0,BigWords,13.32,Man,Man,14.0,1.833333,0.0,86.68
1,1,1016110.0,Conversation,11.01,Man,Man,14.0,1.833333,0.0,88.99
2,2,1016110.0,Drives,11.30,Man,Man,14.0,1.833333,0.0,88.70
3,3,1016110.0,Social,3.67,Man,Man,14.0,1.833333,1.0,96.33
4,4,1016110.0,WPS,-79.67,Man,Man,14.0,1.833333,0.0,179.67


In [18]:
#Lets have a look into our data // Perceived Personalization
pp.head()

,Unnamed: 0,Movez_code_x,Sex,age,FAS,Gender,Linguistic Category,PP value,Similarity: CHOSEN,Similarity: CHOSEN_100
0,0,1016110.0,Man,14.0,1.833333,Man,BigWords,NaN,8.42,91.58
1,1,1016110.0,Man,14.0,1.833333,Man,Conversation,3.0,3.80,96.20
2,2,1016110.0,Man,14.0,1.833333,Man,Drives,4.0,4.08,95.92
3,3,1016110.0,Man,14.0,1.833333,Man,Social,NaN,3.67,96.33
4,4,1016110.0,Man,14.0,1.833333,Man,WPS,2.0,114.34,-14.34


In [19]:
#Lets have a look into our data // Perceived Message Effectiveness
eff.head()

,Unnamed: 0,Movez_code_x,Sex,age,FAS,Gender,Linguistic Category,Eff value,Similarity: CHOSEN,Similarity: CHOSEN_100
0,0,1016110.0,Man,14.0,1.833333,Man,BigWords,NaN,8.42,91.58
1,1,1016110.0,Man,14.0,1.833333,Man,Conversation,2.0,3.80,96.20
2,2,1016110.0,Man,14.0,1.833333,Man,Drives,3.0,4.08,95.92
3,3,1016110.0,Man,14.0,1.833333,Man,Social,NaN,3.67,96.33
4,4,1016110.0,Man,14.0,1.833333,Man,WPS,3.0,114.34,-14.34


In [20]:
# We will drop categories that had too little values overall to be included reliably

dropped_categories = ['WPS', 'lack', 'risk', 'fulfill', 'acquire', 'fatigue', 'need']

complete_df = complete_df[~complete_df['Linguistic Category'].isin(dropped_categories)]
pp = pp[~pp['Linguistic Category'].isin(dropped_categories)]
eff = eff[~eff['Linguistic Category'].isin(dropped_categories)]

### 2. Preprocessing of the Data

In [22]:
#Renaming some columns
pp = pp.rename(columns={"PP value": "PP.value", "Similarity: CHOSEN100": "Similarity_chosen"})
eff = eff.rename(columns={"Eff value": "Eff.value", "Similarity: CHOSEN100": "Similarity_chosen"})


In [23]:
# Recode the 'gender' column: 'Vrouw' becomes 2, 'Man' becomes 1, all others become 0
complete_df['Sex'] = complete_df['Sex'].map({'Vrouw': 2, 'Man': 1}).fillna(0).astype(int)
# Recode the 'gender' column: 'Vrouw' becomes 2, 'Man' becomes 1, all others become 0
pp['Sex'] = pp['Sex'].map({'Vrouw': 2, 'Man': 1}).fillna(0).astype(int)
# Recode the 'gender' column: 'Vrouw' becomes 2, 'Man' becomes 1, all others become 0
eff['Sex'] = eff['Sex'].map({'Vrouw': 2, 'Man': 1}).fillna(0).astype(int)
# Recode the 'gender' column: 'Vrouw' becomes 2, 'Man' becomes 1, all others become 0
#ps['Sex'] = ps['Sex'].map({'Vrouw': 2, 'Man': 1}).fillna(0).astype(int)

In [25]:
# If needed, rename the dependent variable to avoid spaces in the formula
complete_df = complete_df.rename(columns={"Similarity: EXP_100": "Message_similarity"})
complete_df = complete_df.rename(columns={"Linguistic Category": "Linguistic_category"})
eff = eff.rename(columns={"Similarity: CHOSEN_100": "Message_similarity"})
eff = eff.rename(columns={"Linguistic Category": "Linguistic_category"})
pp = pp.rename(columns={"Similarity: CHOSEN_100": "Message_similarity"})
pp = pp.rename(columns={"Linguistic Category": "Linguistic_category"})

# Ensure any categorical predictors (e.g., Sex, Linguistic.Category) are treated as categorical.
complete_df['Sex'] = complete_df['Sex'].astype('category')
complete_df['Linguistic_category'] = complete_df['Linguistic_category'].astype('category')

In [ ]:
complete_df["Linguistic_category"].value_counts()

,count
Linguistic_category,
BigWords,170
Conversation,170
Drives,170
Social,170
allure,170
cogproc,170
curiosity,170
emo_neg,170
emo_pos,170


In [ ]:
# 1. Save DataFrames as a CSV file
complete_df.to_csv("complete_df_collab.csv", index=False)
eff.to_csv("eff_collab.csv", index=False)
pp.to_csv("pp_collab.csv", index=False)

# 2. Use Colab's file download utility
from google.colab import files
#files.download("complete_df_collab.csv")
files.download("eff_collab.csv")
files.download("pp_collab.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 3. Hypothesis 1: Message Similarity predicts Message Preference

In [26]:
## For the analysis we will need to drop missing values

# First, standardize your column names if needed
columns_to_check = [
    "Message_similarity", "age", "Sex", "FAS", "Linguistic_category", "Movez_code_x", "Message_pref"
]

# Count missing values per column
missing_per_column = complete_df[columns_to_check].isnull().sum()

print("❗ Missing values per column:")
print(missing_per_column)

# Drop rows where 'Message_pref' column has missing values
complete_df = complete_df.dropna(subset=['Message_pref'])

❗ Missing values per column:
Message_similarity        0
age                    1260
Sex                       0
FAS                     868
Linguistic_category       0
Movez_code_x              0
Message_pref            294
dtype: int64


First we build a null model and a model with the main predictor and see if there is a significant difference in predicting message preference.

In [ ]:
import pandas as pd
import bambi as bmb
import arviz as az

# Load your data
df = complete_df

# Filter and prepare the dataset
df_model = df[['Message_pref', 'Message_similarity', 'Sex', 'Linguistic_category', 'Movez_code_x']].dropna()
df_model['group1'] = df_model['Linguistic_category'].astype('category')
df_model['group2'] = df_model['Movez_code_x'].astype('category')

# Null model
null_model = bmb.Model(
    "Message_pref ~ Sex + (1|group1) + (1|group2)",
    data=df_model,
    family="bernoulli",

)
null_trace = null_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Full model
full_model = bmb.Model(
    "Message_pref ~ Message_similarity + Sex + (1|group1) + (1|group2)",
    data=df_model,
    family="bernoulli"
)
full_trace = full_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Compare models
az.compare({"null": null_trace, "full": full_trace})

# Posterior summary for Message_similarity
az.summary(full_trace, var_names=["Message_similarity"])

Output()

Output()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Message_similarity,0.023,0.023,-0.019,0.067,0.001,0.0,1118.0,1298.0,1.0


Lets compare the two models and if they significantly differ...

In [ ]:
comp = az.compare({"null": null_trace, "full": full_trace})
print(comp)

      rank     elpd_loo      p_loo  elpd_diff  weight         se       dse  \
full     0 -1394.194501  52.408124   0.000000     1.0  10.710967  0.000000   
null     1 -1394.729680  52.055180   0.535178     0.0  10.554522  0.782556   

      warning scale  
full    False   log  
null    False   log  


There is no significant difference between models in predicting message preference.

Next, we see if the alternative model is effective in predicting Message Preference in a Bayesian Mixed Effects Model

In [ ]:
# Define the model formula.
# This models Message_choice (binary outcome) as a function of Similarity.EXP_z, age, Sex, FAS,
# and includes a random intercept for each level of Linguistic.Category.
formula = "Message_pref ~ Message_similarity + Sex + (1|Linguistic_category) + (1|Movez_code_x)"

# Build the model with a Bernoulli family for logistic regression.
model = bmb.Model(formula, complete_df, family="bernoulli")

# Fit the model (sampling via PyMC under the hood)
results = model.fit()

Output()

In [ ]:
import numpy as np
import arviz as az
from scipy.stats import norm, gaussian_kde

# Make sure `results` is an InferenceData object (e.g., from PyMC, CmdStanPy, or similar)
summary = az.summary(results, round_to=2, kind="stats", stat_focus="mean", hdi_prob=0.95)

# Print the full summary
print(summary.to_string())

                                     mean    sd  hdi_2.5%  hdi_97.5%
1|Linguistic_category[BigWords]     -0.66  0.26     -1.18      -0.19
1|Linguistic_category[Conversation]  0.11  0.20     -0.29       0.50
1|Linguistic_category[Drives]        0.71  0.22      0.29       1.11
1|Linguistic_category[Social]       -0.08  0.29     -0.66       0.49
1|Linguistic_category[allure]       -0.06  0.25     -0.51       0.45
1|Linguistic_category[cogproc]      -0.07  0.28     -0.60       0.51
1|Linguistic_category[curiosity]    -0.59  0.24     -1.11      -0.14
1|Linguistic_category[emo_neg]      -0.50  0.23     -0.96      -0.05
1|Linguistic_category[emo_pos]       0.37  0.22     -0.06       0.78
1|Linguistic_category[intensity]     0.58  0.25      0.08       1.06
1|Linguistic_category[reward]        0.50  0.23      0.07       0.96
1|Linguistic_category[tone_neg]     -0.59  0.23     -1.01      -0.12
1|Linguistic_category[tone_pos]      0.36  0.21     -0.06       0.77
1|Linguistic_category[want]       

Now lets get posterior estiamtes for our main predictor *Message similarity*

In [ ]:
# Extract posterior samples for Similarity.EXP_z.

# Here we assume the parameter is stored in the "posterior" group.
posterior_samples = results.posterior["Similarity.EXP_z"].values.flatten()

# Use kernel density estimation to estimate the posterior density at 0.
kde = gaussian_kde(posterior_samples)
posterior_density_at_0 = kde(0)[0]  # extract scalar

# Assuming a default prior of Normal(0, 2.5) for the coefficient:
prior_sd = 2.5
prior_density_at_0 = norm.pdf(0, loc=0, scale=prior_sd)

# Compute the Savage-Dickey Bayes Factor:
# BF01 is the evidence in favor of the null (parameter = 0).
BF01 = prior_density_at_0 / posterior_density_at_0
# BF10 (the inverse) gives evidence in favor of the alternative.
BF10 = 1 / BF01

print("Prior density at 0:", prior_density_at_0)
print("Posterior density at 0:", posterior_density_at_0)
print("Bayes Factor for Message Similarity as signficant predictor (BF10):", BF10)
print("Bayes Factor against Message Similarity as signficant predictor (BF01):", BF01)

Prior density at 0: 0.15957691216057307
Posterior density at 0: 2.9920401898078595
Bayes Factor in favor of the null model (BF10): 18.749831346511716
Bayes Factor in favor of the null model (BF01): 0.05333381306312622


### 4. Hypothesis 2: Message Similarity predicts Perceived Personalization

In [ ]:
pp.head()

,Unnamed: 0,Movez_code_x,Sex,age,FAS,Gender,Linguistic_category,PP.value,Similarity: CHOSEN,Similarity: CHOSEN_100
0,0,7001593.0,1,17.0,1.833333,Man,emo_pos,5.0,15.09,84.91
1,1,3755626.0,2,16.0,2.000000,Vrouw,emo_pos,5.0,15.31,84.69
2,2,1527216.0,1,16.0,1.166667,"Omschrijf ik liever zelf, namelijk:",emo_pos,1.0,15.08,84.92
3,3,5931444.0,1,16.0,1.333333,Man,emo_pos,1.0,0.96,99.04
4,4,1349968.0,2,16.0,1.500000,Vrouw,emo_pos,5.0,13.33,86.67


Again, we will compare a null model to the alternative model with the main predictor *message similarity*...

We will run this with the Covariate Sex and Age separately to see if they need to be included...

In [27]:
## Model comparison with Age as Covariate

import pandas as pd
import bambi as bmb
import arviz as az

# Load your data
df = pp

# Filter and prepare the dataset
df_model = df[['PP.value', 'age', 'Linguistic_category', 'Movez_code_x']].dropna()
df_model['group1'] = df_model['Linguistic_category'].astype('category')
df_model['group2'] = df_model['Movez_code_x'].astype('category')

# Rename the outcome variable
df_model = df_model.rename(columns={"PP.value": "PP_value"})

# Null model: only random intercepts
null_model = bmb.Model(
    "PP_value ~ (1|group1) + (1|group2)",
    data=df_model,
    family="gaussian",
)
null_trace = null_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Model with Age
age_model = bmb.Model(
    "PP_value ~ age + (1|group1) + (1|group2)",
    data=df_model,
    family="gaussian",
)
age_trace = age_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Compare models
comparison = az.compare({"null": null_trace, "age": age_trace})
print(comparison)

# Posterior summary for Age
az.summary(age_trace, var_names=["age"])


KeyboardInterrupt: 

In [ ]:
## Model comparison with Sex as Covariate

import pandas as pd
import bambi as bmb
import arviz as az

# Load your data
df = pp

# Filter and prepare the dataset
df_model = df[['PP.value', 'Message_similarity', 'Sex', 'Linguistic_category', 'Movez_code_x']].dropna()
df_model['group1'] = df_model['Linguistic_category'].astype('category')
df_model['group2'] = df_model['Movez_code_x'].astype('category')

# Null model
null_model = bmb.Model(
    "PP_value ~ Sex + (1|group1) + (1|group2)",
    data=df_model.rename(columns={"PP.value": "PP_value"}),
    family="gaussian",

)
null_trace = null_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Full model
full_model = bmb.Model(
    "PP_value ~ Message_similarity + Sex + (1|group1) + (1|group2)",
    data=df_model.rename(columns={"PP.value": "PP_value"}),
    family="gaussian"
)
full_trace = full_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Compare models
az.compare({"null": null_trace, "full": full_trace})

# Posterior summary for Message_similarity
az.summary(full_trace, var_names=["Message_similarity"])

Output()

Output()

/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Message_similarity,0.002,0.006,-0.009,0.014,0.0,0.0,1875.0,1423.0,1.0


Lets compare the two models...

In [ ]:
comp = az.compare({"null": null_trace, "full": full_trace})
print(comp)

/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank     elpd_loo       p_loo  elpd_diff    weight         se       dse  \
full     0 -1075.957875  140.221420   0.000000  0.882809  24.407273  0.000000   
null     1 -1076.445901  141.189272   0.488026  0.117191  24.388479  1.140065   

      warning scale  
full     True   log  
null     True   log  


/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


Now we test if the alternative model predicts perceived personailization in a Bayesian Mixed Effects Model...

In [ ]:
# Define the model formula.
# This models Message_choice (binary outcome) as a function of Similarity.EXP_z, age, Sex, FAS,
# and includes a random intercept for each level of Linguistic.Category.
pp = pp[['PP.value', 'Message_similarity', 'Sex', 'Linguistic_category', 'Movez_code_x']].dropna()

formula2 = "PP.value ~ Message_similarity + Sex + (1|Linguistic_category)+ (1|Movez_code_x)"

# Build the model with a Bernoulli family for logistic regression.
model2 = bmb.Model(formula2, pp, family="gaussian")

# Fit the model (sampling via PyMC under the hood)
results2 = model2.fit()

Output()

In [ ]:
# Make sure `results` is an InferenceData object (e.g., from PyMC, CmdStanPy, or similar)
summary2 = az.summary(results2, round_to=2, kind="stats", stat_focus="mean", hdi_prob=0.95)

# Print the full summary
print(summary2.to_string())

                                     mean    sd  hdi_2.5%  hdi_97.5%
1|Linguistic_category[BigWords]     -0.01  0.09     -0.21       0.15
1|Linguistic_category[Conversation]  0.05  0.08     -0.09       0.23
1|Linguistic_category[Drives]       -0.03  0.07     -0.20       0.10
1|Linguistic_category[Social]       -0.02  0.08     -0.22       0.12
1|Linguistic_category[allure]        0.01  0.08     -0.14       0.18
1|Linguistic_category[cogproc]      -0.00  0.10     -0.23       0.21
1|Linguistic_category[curiosity]    -0.02  0.08     -0.20       0.12
1|Linguistic_category[emo_neg]       0.00  0.07     -0.15       0.16
1|Linguistic_category[emo_pos]       0.08  0.08     -0.05       0.25
1|Linguistic_category[intensity]     0.04  0.09     -0.11       0.23
1|Linguistic_category[reward]       -0.07  0.10     -0.28       0.10
1|Linguistic_category[tone_neg]     -0.01  0.08     -0.18       0.15
1|Linguistic_category[tone_pos]     -0.05  0.08     -0.23       0.09
1|Linguistic_category[want]       

Lets also get posterior estimates for Message similarity as main predictor...

In [ ]:
import numpy as np
import arviz as az
from scipy.stats import norm, gaussian_kde

# Extract posterior samples for Similarity.EXP_z.
# Note: The location of the parameter in the InferenceData depends on how your model was specified.
# Here we assume the parameter is stored in the "posterior" group.
posterior_samples = results2.posterior["Similarity_chosen_z"].values.flatten()

# Use kernel density estimation to estimate the posterior density at 0.
kde = gaussian_kde(posterior_samples)
posterior_density_at_0 = kde(0)[0]  # extract scalar

# Assuming a default prior of Normal(0, 2.5) for the coefficient:
prior_sd = 2.5
prior_density_at_0 = norm.pdf(0, loc=0, scale=prior_sd)

# Compute the Savage-Dickey Bayes Factor:
# BF01 is the evidence in favor of the null (parameter = 0).
BF01 = prior_density_at_0 / posterior_density_at_0
# BF10 (the inverse) gives evidence in favor of the alternative.
BF10 = 1 / BF01

print("Prior density at 0:", prior_density_at_0)
print("Posterior density at 0:", posterior_density_at_0)
print("Bayes Factor for Message Similarity as signficant predictor (BF10):", BF10)
print("Bayes Factor against Message Similarity as signficant predictor (BF01):", BF01)

Prior density at 0: 0.15957691216057307
Posterior density at 0: 8.035437178398924
Bayes Factor in favor of the alternative (BF10): 50.35463507598973
Bayes Factor in favor of the null model (BF01): 0.019859145011991628


### 5.Hypothesis 3: Message similarity predicting Perceived Message Effectiveness

In [ ]:
eff.head()

,Unnamed: 0,Movez_code_x,Sex,age,FAS,Gender,Linguistic_category,Eff.value,Similarity: CHOSEN,Message_similarity
0,0,7001593.0,1,17.0,1.833333,Man,emo_pos,5.0,15.09,84.91
1,1,3755626.0,2,16.0,2.000000,Vrouw,emo_pos,1.0,15.31,84.69
2,2,1527216.0,1,16.0,1.166667,"Omschrijf ik liever zelf, namelijk:",emo_pos,1.0,15.08,84.92
3,3,5931444.0,1,16.0,1.333333,Man,emo_pos,5.0,0.96,99.04
4,4,1349968.0,2,16.0,1.500000,Vrouw,emo_pos,3.0,13.33,86.67


In [ ]:
# Again we will remove missing values before the analysis

# First, standardize your column names if needed
columns_to_check = [
    "Message_similarity", "age", "Sex", "FAS", "Linguistic_category", "Movez_code_x", "Eff.value"
]

# Count missing values per column
missing_per_column = eff[columns_to_check].isnull().sum()

print("❗ Missing values per column:")
print(missing_per_column)

# Drop rows with missing values in any of those columns
eff_clean = eff.dropna(subset=columns_to_check)

❗ Missing values per column:
Message_similarity      477
age                    1393
Sex                       0
FAS                     973
Linguistic_category       0
Movez_code_x              0
Eff.value              1833
dtype: int64


In [ ]:
# Ensure any categorical predictors (e.g., Sex, Linguistic.Category) are treated as categorical.
eff['Sex'] = eff['Sex'].astype('category')

In [ ]:
## Lets again compare a null model to the alternative model to predict message effectiveness

import pandas as pd
import bambi as bmb
import arviz as az

# Load your data
df = eff_clean

# Filter and prepare the dataset
df_model = df[['Eff.value', 'Message_similarity', 'Sex', 'Linguistic_category', 'Movez_code_x']].dropna()
df_model['group1'] = df_model['Linguistic_category'].astype('category')
df_model['group2'] = df_model['Movez_code_x'].astype('category')

# Null model
null_model = bmb.Model(
    "Eff_value ~ Sex + (1|group1) + (1|group2)",
    data=df_model.rename(columns={"Eff.value": "Eff_value"}),
    family="gaussian",

)
null_trace = null_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Full model
full_model = bmb.Model(
    "Eff_value ~ Message_similarity + Sex + (1|group1) + (1|group2)",
    data=df_model.rename(columns={"Eff.value": "Eff_value"}),
    family="gaussian"
)
full_trace = full_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Compare models
az.compare({"null": null_trace, "full": full_trace})

# Posterior summary for Message_similarity
az.summary(full_trace, var_names=["Message_similarity"])

Output()

Output()

/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Message_similarity,0.008,0.009,-0.008,0.025,0.0,0.0,1617.0,1142.0,1.0



Lets compare the two models...

In [ ]:
# Lets compare the two models
comp = az.compare({"null": null_trace, "full": full_trace})
print(comp)

/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank    elpd_loo      p_loo  elpd_diff        weight         se  \
null     0 -541.923446  73.627353   0.000000  1.000000e+00  19.161843   
full     1 -543.020795  74.967420   1.097349  1.110223e-16  19.574551   

           dse  warning scale  
null  0.000000     True   log  
full  1.260325     True   log  


/usr/local/lib/python3.11/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


Now lets see how well the alternative model predicts perceived message effectiveness

In [ ]:
# Define the model formula.
# This models Message_choice (binary outcome) as a function of Similarity.EXP_z, age, Sex, FAS,
# and includes a random intercept for each level of Linguistic.Category.
#eff = eff.dropna()#

formula3 = "Eff.value ~ Message_similarity + Sex + (1|Linguistic_category)+ (1|Movez_code_x)"

# Build the model with a Bernoulli family for logistic regression.
model3 = bmb.Model(formula3, eff_clean, family="gaussian")

# Fit the model (sampling via PyMC under the hood)
results3 = model3.fit()

Output()

In [ ]:
import arviz as az

# Make sure `results` is an InferenceData object (e.g., from PyMC, CmdStanPy, or similar)
summary3 = az.summary(results3, round_to=2, kind="stats", stat_focus="mean", hdi_prob=0.95)

# Print the full summary
print(summary3.to_string())

                                     mean    sd  hdi_2.5%  hdi_97.5%
1|Linguistic_category[BigWords]     -0.01  0.21     -0.47       0.38
1|Linguistic_category[Conversation]  0.14  0.18     -0.24       0.47
1|Linguistic_category[Drives]        0.05  0.18     -0.30       0.42
1|Linguistic_category[Social]       -0.34  0.21     -0.75       0.02
1|Linguistic_category[allure]       -0.01  0.18     -0.37       0.36
1|Linguistic_category[cogproc]      -0.03  0.24     -0.54       0.43
1|Linguistic_category[curiosity]     0.34  0.20     -0.04       0.74
1|Linguistic_category[emo_neg]       0.30  0.21     -0.11       0.72
1|Linguistic_category[emo_pos]       0.06  0.13     -0.18       0.36
1|Linguistic_category[intensity]     0.40  0.24     -0.08       0.85
1|Linguistic_category[reward]       -0.42  0.22     -0.84      -0.00
1|Linguistic_category[tone_neg]     -0.05  0.19     -0.39       0.35
1|Linguistic_category[tone_pos]     -0.28  0.21     -0.67       0.09
1|Linguistic_category[want]       

In [ ]:
# Lets get posterior estimates for Lingusitic Similarity as our Main Predictor

# Note: The location of the parameter in the InferenceData depends on how your model was specified.
# Here we assume the parameter is stored in the "posterior" group.
posterior_samples = results3.posterior["Similarity_chosen_z"].values.flatten()

# Use kernel density estimation to estimate the posterior density at 0.
kde = gaussian_kde(posterior_samples)
posterior_density_at_0 = kde(0)[0]  # extract scalar

# Assuming a default prior of Normal(0, 2.5) for the coefficient:
prior_sd = 2.5
prior_density_at_0 = norm.pdf(0, loc=0, scale=prior_sd)

# Compute the Savage-Dickey Bayes Factor:
# BF01 is the evidence in favor of the null (parameter = 0).
BF01 = prior_density_at_0 / posterior_density_at_0
# BF10 (the inverse) gives evidence in favor of the alternative.
BF10 = 1 / BF01

print("Prior density at 0:", prior_density_at_0)
print("Posterior density at 0:", posterior_density_at_0)
print("Bayes Factor for Message Similarity as signficant predictor (BF10):", BF10)
print("Bayes Factor against Message Similarity as signficant predictor (BF01):", BF01)

Prior density at 0: 0.15957691216057307
Posterior density at 0: 6.843519004555879
Bayes Factor in favor of the alternative (BF10): 42.885395586985915
Bayes Factor in favor of the null model (BF01): 0.023317961425158497


### 6. Exploratory Analysis: Message Valence as Potential Moderator of Message Preference

In [ ]:
## Lets first build the different valence groups

positive = ['emo_pos', 'tone_pos', 'Drives', 'reward', 'allure', 'intensitz']
negative = ['emo_neg', 'tone_neg', 'want']
neutral = ['Social', 'Conversation', 'cogproc', 'BigWords', 'allure']

complete_df['valence'] = 'neutral'  # Default value

complete_df.loc[complete_df['Linguistic_category'].isin(positive), 'valence'] = 'positive'
complete_df.loc[complete_df['Linguistic_category'].isin(negative), 'valence'] = 'negative'

In [28]:
#Lets see how much we have

complete_df['valence'].value_counts()

KeyError: 'valence'

As before we will compare a null model to the model with message similarity and valence as predictors

In [ ]:
import pandas as pd
import bambi as bmb
import arviz as az

# Load your data
df = complete_df

# Filter and prepare the dataset
df_model = df[['Message_pref', 'valence', 'Sex', 'Linguistic_category', 'Movez_code_x']].dropna()
df_model['group1'] = df_model['Linguistic_category'].astype('category')
df_model['group2'] = df_model['Movez_code_x'].astype('category')

# Null model
null_model = bmb.Model(
    "Message_pref ~ Sex + (1|group1) + (1|group2)",
    data=df_model,
    family="bernoulli",

)
null_trace = null_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Full model
full_model = bmb.Model(
    "Message_pref ~ valence + Sex + (1|group1) + (1|group2)",
    data=df_model,
    family="bernoulli"
)
full_trace = full_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Compare models
az.compare({"null": null_trace, "full": full_trace})

# Posterior summary for Message_similarity
az.summary(full_trace, var_names=["valence"])

Output()

ERROR:pymc.stats.convergence:There were 23 divergences after tuning. Increase `target_accept` or reparameterize.


Output()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
valence[neutral],0.276,0.302,-0.340,0.826,0.011,0.008,834.0,975.0,1.0
valence[positive],0.760,0.311,0.187,1.322,0.011,0.010,796.0,934.0,1.0


In [ ]:
comp = az.compare({"null": null_trace, "full": full_trace})
print(comp)

      rank     elpd_loo      p_loo  elpd_diff    weight         se      dse  \
null     0 -1394.136692  50.557182   0.000000  0.772452  10.559708  0.00000   
full     1 -1394.354267  50.687866   0.217574  0.227548  10.705930  0.89358   

      warning scale  
null    False   log  
full    False   log  


Now we can run the Bayesian Mixed Effects Models with valence and message similarity*valence as predictors

In [ ]:
# Define the model formula.
# This models Message_choice (binary outcome) as a function of Similarity.EXP_z, age, Sex, FAS,
# and includes a random intercept for each level of Linguistic.Category.

formula4 = "Message_choice ~ Similarity.EXP_z + Similarity.EXP_z:valence + valence + age + Sex + FAS + (1|Linguistic.Category)+ (1|Movez.code_x)"

# Build the model with a Bernoulli family for logistic regression.
model4 = bmb.Model(formula4, complete_df, family="bernoulli")

# Fit the model (sampling via PyMC under the hood)
results4 = model4.fit()

Output()

In [ ]:
# Make sure `results` is an InferenceData object (e.g., from PyMC, CmdStanPy, or similar)
summary4 = az.summary(results4, round_to=2, kind="stats", stat_focus="mean", hdi_prob=0.95)

# Print the full summary
print(summary4.to_string())

                                     mean    sd  hdi_2.5%  hdi_97.5%
1|Linguistic.Category[BigWords]     -0.44  0.28     -0.99       0.07
1|Linguistic.Category[Conversation]  0.18  0.23     -0.27       0.64
1|Linguistic.Category[Drives]        0.29  0.25     -0.15       0.85
1|Linguistic.Category[Social]        0.14  0.29     -0.42       0.74
1|Linguistic.Category[allure]       -0.44  0.39     -1.22       0.27
1|Linguistic.Category[cogproc]       0.11  0.28     -0.40       0.70
1|Linguistic.Category[curiosity]    -0.38  0.27     -0.91       0.16
1|Linguistic.Category[emo_neg]      -0.05  0.28     -0.60       0.52
1|Linguistic.Category[emo_pos]       0.06  0.27     -0.41       0.66
1|Linguistic.Category[intensity]     0.34  0.28     -0.24       0.87
1|Linguistic.Category[reward]        0.16  0.28     -0.39       0.69
1|Linguistic.Category[tone_neg]     -0.30  0.29     -0.93       0.22
1|Linguistic.Category[tone_pos]     -0.04  0.25     -0.53       0.45
1|Linguistic.Category[want]       

Lets get the posterior estimates for message similarity*valence.

In [ ]:
# Extract posterior samples for the interaction between Message similarity and message valence

# Note: The location of the parameter in the InferenceData depends on how your model was specified.
# Here we assume the parameter is stored in the "posterior" group.
posterior_samples = results4.posterior["Similarity.EXP_z:valence"].values.flatten()

# Use kernel density estimation to estimate the posterior density at 0.
kde = gaussian_kde(posterior_samples)
posterior_density_at_0 = kde(0)[0]  # extract scalar

# Assuming a default prior of Normal(0, 2.5) for the coefficient:
prior_sd = 2.5
prior_density_at_0 = norm.pdf(0, loc=0, scale=prior_sd)

# Compute the Savage-Dickey Bayes Factor:
# BF01 is the evidence in favor of the null (parameter = 0).
BF01 = prior_density_at_0 / posterior_density_at_0
# BF10 (the inverse) gives evidence in favor of the alternative.
BF10 = 1 / BF01

print("Prior density at 0:", prior_density_at_0)
print("Posterior density at 0:", posterior_density_at_0)
print("Bayes Factor in favor of the alternative (BF10):", BF10)

### 7. Exploratory Analysis: Message Similarity across all measure predicts perceived source similarity

In [ ]:
ps.head()

,Movez_code_x,Sex,Gender,age,FAS,Message_choice_mean,Chosen_message_mean,Similarity_mean,Chosen_message_mean100
0,7001593.0,1,Man,17.0,1.833333,0.611111,8.775000,3.0,91.225000
1,3755626.0,2,Vrouw,16.0,2.000000,0.500000,8.796111,2.5,91.203889
2,1527216.0,1,"Omschrijf ik liever zelf, namelijk:",16.0,1.166667,0.333333,6.142778,3.0,93.857222
3,5931444.0,1,Man,16.0,1.333333,0.444444,6.391667,1.0,93.608333
4,1349968.0,2,Vrouw,16.0,1.500000,0.666667,9.496111,3.0,90.503889


In [ ]:
import pandas as pd
import bambi as bmb
import arviz as az

# Drop missing values
df_model = ps[['Similarity_mean', 'Sex', 'Chosen_message_mean100']].dropna()

# Fit the null model (only Sex)
null_model = bmb.Model("Similarity_mean ~ Sex", data=df_model, family="gaussian")
null_trace = null_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Fit the full model (Sex + Chosen_message_mean100)
full_model = bmb.Model("Similarity_mean ~ Sex + Chosen_message_mean100", data=df_model, family="gaussian")
full_trace = full_model.fit(
    draws=1000,
    tune=1000,
    idata_kwargs={"log_likelihood": True}
)

# Compare models using LOO
comparison = az.compare({"null": null_trace, "full": full_trace})
print(comparison)

# Posterior summary for Chosen_message_mean100
summary = az.summary(full_trace, var_names=["Chosen_message_mean100"])
print(summary)

Output()

Output()

      rank    elpd_loo     p_loo  elpd_diff   weight        se       dse  \
null     0 -278.314761  2.950891   0.000000  0.61145  8.542617  0.000000   
full     1 -278.432829  3.504489   0.118068  0.38855  8.438699  0.993769   

      warning scale  
null    False   log  
full    False   log  
                         mean    sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
Chosen_message_mean100  0.023  0.02  -0.018    0.057        0.0      0.0   

                        ess_bulk  ess_tail  r_hat  
Chosen_message_mean100    3108.0    1436.0    1.0  
